##Import statements

In [ ]:
!pip install transformers datasets faiss-cpu psutil

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 90.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 22.8 MB/s eta 0:00:00


In [ ]:
import logging
import os
from dataclasses import dataclass, field
from functools import partial
from pathlib import Path
from tempfile import TemporaryDirectory
from typing import List, Optional
import json
import csv
import pandas as pd

import torch
from datasets import Features, Sequence, Value, load_dataset

import faiss
from transformers import (
    DPRContextEncoder,
    DPRContextEncoderTokenizerFast,
    HfArgumentParser,
    RagRetriever,
    RagSequenceForGeneration,
    RagTokenizer,
)

import nltk
from nltk.tokenize import word_tokenize
from tqdm import tqdm


nltk.download('punkt')

dataset_dir = Path('squad')
if not dataset_dir.is_dir():
  !mkdir squad
  !wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json -O squad/train-v1.1.json
  !wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json -O squad/dev-v1.1.json


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


--2023-11-28 17:01:31--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30288272 (29M) [application/json]
Saving to: ‘squad/train-v1.1.json’

squad/train-v1.1.js 100%[===================>]  28.88M  --.-KB/s    in 0.08s   

2023-11-28 17:01:31 (380 MB/s) - ‘squad/train-v1.1.json’ saved [30288272/30288272]

--2023-11-28 17:01:31--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4854279 (4.6M) [application/json]
Saving to: ‘squad/dev-v1.1.json’

squ

In [ ]:
!pip install rank_bm25
from rank_bm25 import BM25Okapi


In [ ]:
def convert(inputfile, outputfile):
    with open(inputfile, 'r') as json_file:
        json_object = json.load(json_file)
    header = ['id', 'question','title', 'context_id', 'context']
    with open(outputfile, 'w', encoding="utf-8", newline='') as file:
        writer = csv.writer(file)
        writer.writerow(header)

        context_id = 0
        previous_context = json_object['data'][0]['paragraphs'][0]['context']

        for i in range (len(json_object['data'])):
            for j in range(len(json_object['data'][i]['paragraphs'])):
                for k in range(len(json_object['data'][i]['paragraphs'][j]['qas'])):
                    row = []
                    row.append(json_object['data'][i]['paragraphs'][j]['qas'][k]['id'])
                    row.append(json_object['data'][i]['paragraphs'][j]['qas'][k]['question'])
                    row.append(json_object['data'][i]['title'])

                    current_context = json_object['data'][i]['paragraphs'][j]['context']
                    if previous_context != current_context:
                      context_id += 1
                      previous_context = current_context
                    row.append(context_id)
                    row.append(current_context)
                    if (len(row) != 0):
                        writer.writerow(row)

convert(f'{dataset_dir}/dev-v1.1.json', f'{dataset_dir}/dev.csv')
convert(f'{dataset_dir}/train-v1.1.json', f'{dataset_dir}/train.csv')

raw_dataset = load_dataset('csv', data_files={'train':f'{dataset_dir}/train.csv', 'validation':f'{dataset_dir}/dev.csv'})
train_df = pd.read_csv(f'{dataset_dir}/train.csv')
dev_df = pd.read_csv(f'{dataset_dir}/dev.csv')

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [ ]:
train_df

,id,question,title,context_id,context
0,5733be284776f41900661182,To whom did the Virgin Mary allegedly appear i...,University_of_Notre_Dame,0,"Architecturally, the school has a Catholic cha..."
1,5733be284776f4190066117f,What is in front of the Notre Dame Main Building?,University_of_Notre_Dame,0,"Architecturally, the school has a Catholic cha..."
2,5733be284776f41900661180,The Basilica of the Sacred heart at Notre Dame...,University_of_Notre_Dame,0,"Architecturally, the school has a Catholic cha..."
3,5733be284776f41900661181,What is the Grotto at Notre Dame?,University_of_Notre_Dame,0,"Architecturally, the school has a Catholic cha..."
4,5733be284776f4190066117e,What sits on top of the Main Building at Notre...,University_of_Notre_Dame,0,"Architecturally, the school has a Catholic cha..."
...,...,...,...,...,...
87594,5735d259012e2f140011a09d,In what US state did Kathmandu first establish...,Kathmandu,18894,"Kathmandu Metropolitan City (KMC), in order to..."
87595,5735d259012e2f140011a09e,What was Yangon previously known as?,Kathmandu,18894,"Kathmandu Metropolitan City (KMC), in order to..."
87596,5735d259012e2f140011a09f,With what Belorussian city does Kathmandu have...,Kathmandu,18894,"Kathmandu Metropolitan City (KMC), in order to..."
87597,5735d259012e2f140011a0a0,In what year did Kathmandu create its initial ...,Kathmandu,18894,"Kathmandu Metropolitan City (KMC), in order to..."


In [ ]:
dev_df

,id,question,title,context_id,context
0,56be4db0acb8001400a502ec,Which NFL team represented the AFC at Super Bo...,Super_Bowl_50,0,Super Bowl 50 was an American football game to...
1,56be4db0acb8001400a502ed,Which NFL team represented the NFC at Super Bo...,Super_Bowl_50,0,Super Bowl 50 was an American football game to...
2,56be4db0acb8001400a502ee,Where did Super Bowl 50 take place?,Super_Bowl_50,0,Super Bowl 50 was an American football game to...
3,56be4db0acb8001400a502ef,Which NFL team won Super Bowl 50?,Super_Bowl_50,0,Super Bowl 50 was an American football game to...
4,56be4db0acb8001400a502f0,What color was used to emphasize the 50th anni...,Super_Bowl_50,0,Super Bowl 50 was an American football game to...
...,...,...,...,...,...
10565,5737aafd1c456719005744fb,What is the metric term less used than the New...,Force,2066,"The pound-force has a metric counterpart, less..."
10566,5737aafd1c456719005744fc,What is the kilogram-force sometimes reffered ...,Force,2066,"The pound-force has a metric counterpart, less..."
10567,5737aafd1c456719005744fd,What is a very seldom used unit of mass in the...,Force,2066,"The pound-force has a metric counterpart, less..."
10568,5737aafd1c456719005744fe,What seldom used term of a unit of force equal...,Force,2066,"The pound-force has a metric counterpart, less..."


In [ ]:
#dev_df
question_df2 = dev_df["question"]
question2 = question_df2.values.tolist()
context_df2 = dev_df["context"].drop_duplicates()
context2 = context_df2.values.tolist()

print("# of questions: ", len(question2))
print("# of contexts: ", len(context2))

# of questions:  10570
# of contexts:  2067


In [ ]:
dev_df

,id,question,title,context_id,context
0,56be4db0acb8001400a502ec,Which NFL team represented the AFC at Super Bo...,Super_Bowl_50,0,Super Bowl 50 was an American football game to...
1,56be4db0acb8001400a502ed,Which NFL team represented the NFC at Super Bo...,Super_Bowl_50,0,Super Bowl 50 was an American football game to...
2,56be4db0acb8001400a502ee,Where did Super Bowl 50 take place?,Super_Bowl_50,0,Super Bowl 50 was an American football game to...
3,56be4db0acb8001400a502ef,Which NFL team won Super Bowl 50?,Super_Bowl_50,0,Super Bowl 50 was an American football game to...
4,56be4db0acb8001400a502f0,What color was used to emphasize the 50th anni...,Super_Bowl_50,0,Super Bowl 50 was an American football game to...
...,...,...,...,...,...
10565,5737aafd1c456719005744fb,What is the metric term less used than the New...,Force,2066,"The pound-force has a metric counterpart, less..."
10566,5737aafd1c456719005744fc,What is the kilogram-force sometimes reffered ...,Force,2066,"The pound-force has a metric counterpart, less..."
10567,5737aafd1c456719005744fd,What is a very seldom used unit of mass in the...,Force,2066,"The pound-force has a metric counterpart, less..."
10568,5737aafd1c456719005744fe,What seldom used term of a unit of force equal...,Force,2066,"The pound-force has a metric counterpart, less..."


In [ ]:
# id = dev_df["context_id"][10568]
# cont = context2[id]
# print(cont)

The pound-force has a metric counterpart, less commonly used than the newton: the kilogram-force (kgf) (sometimes kilopond), is the force exerted by standard gravity on one kilogram of mass. The kilogram-force leads to an alternate, but rarely used unit of mass: the metric slug (sometimes mug or hyl) is that mass that accelerates at 1 m·s−2 when subjected to a force of 1 kgf. The kilogram-force is not a part of the modern SI system, and is generally deprecated; however it still sees use for some purposes as expressing aircraft weight, jet thrust, bicycle spoke tension, torque wrench settings and engine output torque. Other arcane units of force include the sthène, which is equivalent to 1000 N, and the kip, which is equivalent to 1000 lbf.


##Tokenize contexts


In [ ]:
ctx_tokenizer = DPRContextEncoderTokenizerFast.from_pretrained("facebook/dpr-ctx_encoder-multiset-base")

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/492 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizerFast'.


In [ ]:
# context = raw_dataset["train"]["context"]
# title = raw_dataset["train"]["title"]
question_df = train_df["question"]
question = question_df.values.tolist()
context_df = train_df["context"].drop_duplicates()
context = context_df.values.tolist()
# title_df = train_df["title"]
# title = title_df.values.tolist()

print("# of questions: ", len(question))
# print("# of titles: ", len(title))
print("# of contexts: ", len(context))

# of questions:  87599
# of contexts:  18891


In [ ]:
outputs = ctx_tokenizer(context, truncation=True, padding = "longest", return_tensors = "pt")
input_ids= outputs["input_ids"] #holds the encoded tokens
print(input_ids.shape)

In [ ]:
#for validation set
outputs_v = ctx_tokenizer(context2, truncation=True, padding = "longest", return_tensors = "pt")
input_ids_v= outputs_v["input_ids"] #holds the encoded tokens
print(input_ids_v.shape)

torch.Size([2067, 512])


##Encode contexts

In [ ]:
if torch.cuda.is_available():
  device= torch.device("cuda")
  print("There are %d GPU(s) available." %torch.cuda.device_count())

  print("We will use the GPU", torch.cuda.get_device_name(0))
else:
  device = torch.device("cpu")
  print("No GPU available.")

There are 1 GPU(s) available.
We will use the GPU Tesla T4


In [ ]:
ctx_encoder = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-multiset-base")
ctx_encoder = ctx_encoder.to(device=device)

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/dpr-ctx_encoder-multiset-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.weight', 'ctx_encoder.bert_model.pooler.dense.bias']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
#encoding is slow, so let's print progress updates
import time
import datetime
import math
def format_time(elapsed):
  elapsed_rounded = int(round(elapsed))
  return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
import math

#we are running the model forward only, so no need for gradients
torch.set_grad_enabled(False)
t0 =time.time()
step =0

batch_size = 64
num_contexts = input_ids.size()[0]

num_batches = math.ceil(num_contexts/batch_size)

#as we embed the passages in batches, accumulate them in this list
embeds_batches =[]

print("Generating embeddings for {} passages... ".format(num_contexts))

for i in range(0, num_contexts, batch_size):
  if step %100 == 0 and step!=0:
    elapsed = format_time(time.time()-t0)
    #progress ??
    print("Batch {:>5,} of {:>5,}.         Elapsed: {:}".format(step, num_batches, elapsed))
  #next_batch
  batch_ids = input_ids[i:i+64, :]
  batch_ids = batch_ids.to(device)

  outputs = ctx_encoder(batch_ids, return_dict=True)
  embeddings = outputs["pooler_output"] #embeddings are in this field

  #bring the embeddings back over from gpu & convert to numpy (from pytorch)
  embeddings = embeddings.detach().cpu().numpy()

  embeds_batches.append(embeddings)
  step +=1

print("Done.")



In [ ]:
#for validation set
#we are running the model forward only, so no need for gradients
torch.set_grad_enabled(False)
t0 =time.time()
step =0

batch_size = 64
num_contexts_v = input_ids_v.size()[0]

num_batches = math.ceil(num_contexts_v/batch_size)

#as we embed the passages in batches, accumulate them in this list
embeds_batches_v =[]

print("Generating embeddings for {} passages... ".format(num_contexts_v))

for i in range(0, num_contexts_v, batch_size):
  if step %100 == 0 and step!=0:
    elapsed = format_time(time.time()-t0)
    #progress ??
    print("Batch {:>5,} of {:>5,}.         Elapsed: {:}".format(step, num_batches, elapsed))
  #next_batch
  batch_ids = input_ids_v[i:i+64, :]
  batch_ids = batch_ids.to(device)

  outputs = ctx_encoder(batch_ids, return_dict=True)
  embeddings = outputs["pooler_output"] #embeddings are in this field

  #bring the embeddings back over from gpu & convert to numpy (from pytorch)
  embeddings = embeddings.detach().cpu().numpy()

  embeds_batches_v.append(embeddings)
  step +=1

print("Done.")


Generating embeddings for 2067 passages... 
Done.


In [ ]:
import numpy as np

In [ ]:

#combine results across all batches.
embeddings =np.concatenate(embeds_batches, axis=0)

print("Size of dataset embeddings: ", embeddings.shape)

Size of dataset embeddings:  (18891, 768)


In [ ]:
#for validation set
#combine results across all batches.
embeddings_v =np.concatenate(embeds_batches_v, axis=0)

print("Size of dataset embeddings: ", embeddings_v.shape)

Size of dataset embeddings:  (2067, 768)


##FAISS Index

In [ ]:
#the dimension of the embeddings to pass to the hnsw faiss index
dim=768

#the number of bidirectional links created for every new element during HNSW indext construction
m=128

#fast approximate nearest neighbor search using Faiss implementation of HNSW
index = faiss.IndexHNSWFlat(dim, m, faiss.METRIC_INNER_PRODUCT)

In [ ]:
#for validation set
#fast approximate nearest neighbor search using Faiss implementation of HNSW
index_v = faiss.IndexHNSWFlat(dim, m, faiss.METRIC_INNER_PRODUCT)

In [ ]:
print("Building the FAISS index... ")

t0=time.time()

index.train(embeddings)

index.add(embeddings)

print("   Done.")

print("   Adding the embeddings to index took", format_time(time.time()-t0))


Building the FAISS index... 
   Done.
   Adding the embeddings to index took 0:00:08


In [ ]:
print("Building the FAISS index... ")

t0=time.time()

index_v.train(embeddings_v)

index_v.add(embeddings_v)

print("   Done.")

print("   Adding the embeddings to index took", format_time(time.time()-t0))

Building the FAISS index... 
   Done.
   Adding the embeddings to index took 0:00:00


##Search (realized after writing that this section isn't necessary except for import statements, so just skip)

DPR question tokenizer + encoder

In [ ]:
from transformers.models.dpr.modeling_dpr import DPRQuestionEncoder
q_encoder = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-multiset-base")

q_encoder =q_encoder.to(device=device)

config.json:   0%|          | 0.00/493 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/dpr-question_encoder-multiset-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from transformers.models.dpr.tokenization_dpr_fast import DPRQuestionEncoderTokenizerFast
q_tokenizer = DPRQuestionEncoderTokenizerFast.from_pretrained("facebook/dpr-question_encoder-multiset-base")

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
outputs2 = q_tokenizer(question,truncation=True, padding = "longest", return_tensors="pt")
input_ids2= outputs2["input_ids"] #holds the encoded tokens
print(input_ids2.shape)

torch.Size([87599, 63])


In [ ]:

#we are running the model forward only, so no need for gradients
torch.set_grad_enabled(False)
t0 =time.time()
step =0

batch_size = 64
num_contexts2 = input_ids2.size()[0]

num_batches2 = math.ceil(num_contexts2/batch_size)

#as we embed the passages in batches, accumulate them in this list
embeds_batches2 =[]

print("Generating embeddings for {} questions... ".format(num_contexts2))

for i in range(0, num_contexts2, batch_size):
  if step %100 == 0 and step!=0:
    elapsed = format_time(time.time()-t0)
    #progress ??
    print("Batch {:>5,} of {:>5,}.         Elapsed: {:}".format(step, num_batches2, elapsed))
  #next_batch
  batch_ids = input_ids2[i:i+64, :]
  batch_ids = batch_ids.to(device)

  outputs = q_encoder(batch_ids, return_dict=True)
  embeddings = outputs["pooler_output"] #embeddings are in this field

  #bring the embeddings back over from gpu & convert to numpy (from pytorch)
  embeddings = embeddings.detach().cpu().numpy()

  embeds_batches2.append(embeddings)
  step +=1

print("Done.")

Generating embeddings for 87599 questions... 
Batch   100 of   296.         Elapsed: 0:00:25
Batch   200 of   296.         Elapsed: 0:00:49
Batch   300 of   296.         Elapsed: 0:01:13
Batch   400 of   296.         Elapsed: 0:01:37
Batch   500 of   296.         Elapsed: 0:02:01
Batch   600 of   296.         Elapsed: 0:02:26
Batch   700 of   296.         Elapsed: 0:02:50
Batch   800 of   296.         Elapsed: 0:03:14
Batch   900 of   296.         Elapsed: 0:03:38
Batch 1,000 of   296.         Elapsed: 0:04:02
Batch 1,100 of   296.         Elapsed: 0:04:27
Batch 1,200 of   296.         Elapsed: 0:04:51
Batch 1,300 of   296.         Elapsed: 0:05:15
Done.


In [ ]:
embeddings2 =np.concatenate(embeds_batches2, axis=0)

print("Size of dataset embeddings: ", embeddings2.shape)


Size of dataset embeddings:  (87599, 768)


##Example Test

In [ ]:
D, I = index.search(embeddings2[1].reshape(-1, 768), k=10)

print("closest matching indeces: ", I)
print("Inner Product: ", D)

closest matching indeces:  [[14202    38  3721 12630    52     7 18356  8476 14201 10613]]
Inner Product:  [[72.532005 70.6931   69.273926 68.67923  68.50221  68.271095 68.22084
  67.90174  67.78172  67.64709 ]]


In [ ]:
query = dev_df["question"][10568]
input_ids_q= q_tokenizer.encode(query, return_tensors="pt")
input_ids_q=input_ids_q.to(device)
outputs_q=q_encoder(input_ids_q)
q_emb = outputs_q["pooler_output"]
q_emb=q_emb.cpu().numpy()
D, I = index_v.search(q_emb, k=5)

ans = []
for i in I[0]:
  ans.append(context2[i])
print("closest matching indeces: ", I)
print("Inner Product: ", D)
for el in ans:
  print(el)
  print('\n')

closest matching indeces:  [[2066 2048 2056 2030 2034]]
Inner Product:  [[83.95572 72.33777 70.95131 70.11519 69.93923]]
The pound-force has a metric counterpart, less commonly used than the newton: the kilogram-force (kgf) (sometimes kilopond), is the force exerted by standard gravity on one kilogram of mass. The kilogram-force leads to an alternate, but rarely used unit of mass: the metric slug (sometimes mug or hyl) is that mass that accelerates at 1 m·s−2 when subjected to a force of 1 kgf. The kilogram-force is not a part of the modern SI system, and is generally deprecated; however it still sees use for some purposes as expressing aircraft weight, jet thrust, bicycle spoke tension, torque wrench settings and engine output torque. Other arcane units of force include the sthène, which is equivalent to 1000 N, and the kip, which is equivalent to 1000 lbf.


In this equation, a dimensional constant  is used to describe the relative strength of gravity. This constant has come to be kn

In [ ]:
# import textwrap

# wrapper = textwrap.TextWrapper()

# for i in I[0]:
#   print("Index: ", i)

#   c = context2[i]

#   print("Context: ")
#   print(wrapper.fill(c))

#   print('')

Index:  0
Context: 
Super Bowl 50 was an American football game to determine the champion
of the National Football League (NFL) for the 2015 season. The
American Football Conference (AFC) champion Denver Broncos defeated
the National Football Conference (NFC) champion Carolina Panthers
24–10 to earn their third Super Bowl title. The game was played on
February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at
Santa Clara, California. As this was the 50th Super Bowl, the league
emphasized the "golden anniversary" with various gold-themed
initiatives, as well as temporarily suspending the tradition of naming
each Super Bowl game with Roman numerals (under which the game would
have been known as "Super Bowl L"), so that the logo could prominently
feature the Arabic numerals 50.

Index:  53
Context: 
Super Bowl 50 featured numerous records from individuals and teams.
Denver won despite being massively outgained in total yards (315 to
194) and first downs (21 to 11). Their 194 yar

In [ ]:
import textwrap

wrapper = textwrap.TextWrapper()

for i in I[0]:
  print("Index: ", i)

  c = context[i]

  print("Context: ")
  print(wrapper.fill(c))

  print('')

Index:  14202
Context: 
The centre of Paris contains the most visited monuments in the city,
including the Notre Dame Cathedral and the Louvre as well as the
Sainte-Chapelle; Les Invalides, where the tomb of Napoleon is located,
and the Eiffel Tower are located on the Left Bank south-west of the
centre. The banks of the Seine from the Pont de Sully to the Pont
d'Iéna have been listed as a UNESCO World Heritage Site since 1991.
Other landmarks are laid out east to west along the historic axis of
Paris, which runs from the Louvre through the Tuileries Garden, the
Luxor Column in the Place de la Concorde, the Arc de Triomphe, to the
Grande Arche of La Défense.

Index:  38
Context: 
The University of Notre Dame du Lac (or simply Notre Dame
/ˌnoʊtərˈdeɪm/ NOH-tər-DAYM) is a Catholic research university located
adjacent to South Bend, Indiana, in the United States. In French,
Notre Dame du Lac means "Our Lady of the Lake" and refers to the
university's patron saint, the Virgin Mary. The main

##Top-k retrieval accuracy using train set


In [ ]:
def find_topk_acc(k):
  success_num = 0
  for row_id in tqdm(train_df.index, mininterval = 3, desc ="Evaluating..."):

    query = train_df["question"][row_id]
    gold_context = train_df["context"][row_id]

    #process query
    input_ids_q= q_tokenizer.encode(query, return_tensors="pt")
    input_ids_q=input_ids_q.to(device)
    outputs_q=q_encoder(input_ids_q)
    q_emb = outputs_q["pooler_output"]
    q_emb=q_emb.cpu().numpy()

      # Get the ids and similarity scores of top k context
    D, I = index.search(q_emb, k=k)
    ans =[]
    for i in I[0]:
      ans.append(context[i])

    if gold_context in ans:
      success_num += 1
    topk_acc = (success_num/len(train_df)) * 100

  print(f"Successful retrievals: {success_num}/{len(dev_df)}")
  return topk_acc

print(f'Top-k retrieval accuracy of DPR with k=5: {find_topk_acc(5)}')

Evaluating...: 100%|██████████| 87599/87599 [20:14<00:00, 72.11it/s]

Successful retrievals: 54285/10570
Top-k retrieval accuracy of BM25 with k=5: 61.96988550097604


In [ ]:
print(f'Top-k retrieval accuracy of DPR with k=20: {find_topk_acc(20)}')

Evaluating...: 100%|██████████| 87599/87599 [19:56<00:00, 73.22it/s]

Successful retrievals: 67310/10570
Top-k retrieval accuracy of DPR with k=20: 76.83877669836414


##Top-k retrieval accuracy using validation set

In [ ]:
def find_topk_acc_v(k):
  success_num = 0
  for row_id in tqdm(dev_df.index, mininterval = 3, desc ="Evaluating..."):

    query = dev_df["question"][row_id]
    gold_context = dev_df["context"][row_id]

    #process query
    input_ids_q= q_tokenizer.encode(query, return_tensors="pt")
    input_ids_q=input_ids_q.to(device)
    outputs_q=q_encoder(input_ids_q)
    q_emb = outputs_q["pooler_output"]
    q_emb=q_emb.cpu().numpy()

      # Get the ids and similarity scores of top k context
    D, I = index_v.search(q_emb, k=k)
    ans =[]
    for i in I[0]:
      ans.append(context2[i])

    if gold_context in ans:
      success_num += 1
    topk_acc = (success_num/len(dev_df)) * 100

  print(f"Successful retrievals: {success_num}/{len(dev_df)}")
  return topk_acc

print(f'Top-k retrieval accuracy of DPR with k=5: {find_topk_acc_v(5)}')
print(f'Top-k retrieval accuracy of DPR with k=20: {find_topk_acc_v(20)}')

Evaluating...: 100%|██████████| 10570/10570 [02:33<00:00, 69.05it/s]


Successful retrievals: 7892/10570
Top-k retrieval accuracy of DPR with k=5: 74.66414380321666


Evaluating...: 100%|██████████| 10570/10570 [02:20<00:00, 75.32it/s]

Successful retrievals: 9489/10570
Top-k retrieval accuracy of DPR with k=20: 89.77294228949859


##Applying BM25 on SQUAD validation dataset


In [ ]:
tokenized_corpus = [word_tokenize(doc) for doc in context2]
bm25 = BM25Okapi(tokenized_corpus)

In [ ]:
def find_topk_acc_BM25(k):
  success_num = 0
  for row_id in tqdm(dev_df.index, mininterval = 3, desc ="Evaluating..."):
    query = dev_df["question"][row_id]
    gold_context_id = dev_df["context_id"][row_id]

    tokenized_query = word_tokenize(query)
    # Get relevance score for each context
    context_scores = torch.tensor(bm25.get_scores(tokenized_query))
    # Get the context id of top ten context
    _, top_context_id = torch.topk(context_scores, k=k)

    if gold_context_id in top_context_id:
      success_num += 1
    topk_acc = (success_num/len(dev_df)) * 100

  print(f"Successful retrievals: {success_num}/{len(dev_df)}")
  return topk_acc

print(f'Top-k retrieval accuracy of BM25 with k=5: {find_topk_acc_BM25(5)}')
# print(f'Top-k retrieval accuracy of BM25 with k=20: {find_topk_acc_BM25(20)}')

Evaluating...: 100%|██████████| 10570/10570 [01:56<00:00, 90.71it/s] 

Successful retrievals: 9192/10570
Top-k retrieval accuracy of BM25 with k=5: 86.96310312204352


##Combining results of DPR and BM25 with linear combination


In [ ]:
def combine_rankings_linear_normalized(dpr_ranking, bm25_ranking, dpr_ids, bm25_ids, alpha):
    combined_ranking = []
    k = len(dpr_ranking)
    min_dpr = min(dpr_ranking)
    for passage_id in dpr_ids:
        if passage_id in bm25_ids:
            bm25_score = bm25_ranking[bm25_ids.index(passage_id)]
        else:
            # If not in BM25 ranking, assign it the minimum DPR score
            bm25_score = min_dpr

        dpr_score = dpr_ranking[dpr_ids.index(passage_id)]
        combined_score = alpha * dpr_score + bm25_score
        combined_ranking.append((passage_id, combined_score))

    # Add passages from BM25 ranking not present in DPR
    for passage_id in bm25_ids:
        if passage_id not in dpr_ids:
            dpr_score = min_dpr
            bm25_score = bm25_ranking[bm25_ids.index(passage_id)]
            combined_score = alpha * dpr_score + bm25_score
            combined_ranking.append((passage_id, combined_score))

    combined_ranking = sorted(combined_ranking, key=lambda x: x[1], reverse=True)
    return combined_ranking[:k]



In [ ]:
def find_topk_acc_combined_validation(k):
  success_num = 0
  for row_id in tqdm(dev_df.index, mininterval = 3, desc ="Evaluating..."):

    query = dev_df["question"][row_id]
    gold_context = dev_df["context"][row_id]
    gold_context_id = dev_df["context_id"][row_id]

    #process query for DPR --------------------------------------------------------
    input_ids_q= q_tokenizer.encode(query, return_tensors="pt")
    input_ids_q=input_ids_q.to(device)
    outputs_q=q_encoder(input_ids_q)
    q_emb = outputs_q["pooler_output"]
    q_emb=q_emb.cpu().numpy()

    # Get the ids and similarity scores of top k context
    D, I = index_v.search(q_emb, k=k)
    ans_context =[]  # ranking contexts of DPR
    ans_scores= D[0].tolist()# ranking scores of DPR
    ans_ids = I[0].tolist() #ranking ids of DPR
    for i in I[0]:
      ans_context.append(context2[i])
    #------------------------------------------------------------
    #get results from bm25
    tokenized_query = word_tokenize(query)
    # Get relevance score for each context
    context_scores = torch.tensor(bm25.get_scores(tokenized_query))
    # Get the context id of top ten context
    top_context_scores, top_context_id = torch.topk(context_scores, k=k)
    top_context_id = top_context_id.tolist()
    top_context_scores= top_context_scores.tolist()
    bm25_ans_context =[]
    for i in top_context_id:
      bm25_ans_context.append(context2[i])
    #----------------------------------------------------------------------------------
    comb_ans = combine_rankings_linear_normalized(ans_scores, top_context_scores, ans_ids, top_context_id, 1.1) #i took the value of alpha from the paper
    comb_ids=[]
    for el in comb_ans:
      comb_ids.append(el[0])
    if gold_context_id in comb_ids:
      success_num += 1

    topk_acc = (success_num/len(dev_df)) * 100

  print(f"Successful retrievals: {success_num}/{len(dev_df)}")
  return topk_acc

print(f'Top-k retrieval accuracy of DPR +BM25 with k=5: {find_topk_acc_combined_validation(5)}')

Evaluating...: 100%|██████████| 10570/10570 [04:08<00:00, 42.56it/s]

Successful retrievals: 8355/10570
Top-k retrieval accuracy of DPR +BM25 with k=5: 79.04446546830653


In [ ]:
print(f'Top-k retrieval accuracy of DPR +BM25 with k=20: {find_topk_acc_combined_validation(20)}')

Evaluating...: 100%|██████████| 10570/10570 [04:00<00:00, 43.97it/s]

Successful retrievals: 9883/10570
Top-k retrieval accuracy of DPR +BM25 with k=20: 93.50047303689688
